<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/homework-6-hoonheelee0202/blob/master/kafka_producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install kafka-python
!pip3 install pickle5
!pip3 install geopandas
!pip3 install jsonpickle
!pip3 install jsonlines

     |████████████████████████████████| 256kB 6.1MB/s 
     |████████████████████████████████| 133kB 6.0MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219247 sha256=b80647d76d0b16e24dce4ecab867eb23db754c12ded9492d9e0a5252321bc266
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
     |████████████████████████████████| 1.0MB 6.2MB/s 
     |████████████████████████████████| 6.5MB 17.7MB/s 
     |████████████████████████████████| 15.3MB 303kB/s 


In [2]:
# upload nashville-tweets-2019-01-28.zip, Salaries.csv.zip, Batting.csv.zip
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving incident_NFD_XDSegID_BigDataClass.pkl to incident_NFD_XDSegID_BigDataClass.pkl
User uploaded file "incident_NFD_XDSegID_BigDataClass.pkl" with length 4381353 bytes


In [6]:
# Server IP
server_ip = '34.204.95.129'

In [8]:
import time
from kafka import KafkaProducer
import pickle5 as pickle

'''
Write your Kafka producer that reads samples from data file bylines,
then publishes to the the Kafka server. 
'''
def myProducer(bootstrap_server, data_file_path, topic):
  # TODO: implement a kafka producer
  # You may use sleep function to slow the publication rate, but it's not required

  # Declare Server Address Variable
  kafka_server_addr = bootstrap_server+":9092"
  print("kafka_server_addr: ", kafka_server_addr)
  
  try:
    # Declare the producer
    producer = KafkaProducer(bootstrap_servers=[kafka_server_addr])
  except:
    print("producer error")


  # Read the file
  file_incident = open(data_file_path,'rb')
  incident_pickle=pickle.load(file_incident)
  

  # Drop unnecessary field - ID_Original
  json_data = incident_pickle.drop(['ID_Original'], axis=1)

  # fill NaN as 0
  json_data = json_data.fillna(0)

  count = 0
  for i in range(len(json_data.index)):
    send_data="{"+"\"latitude\":{},\"longitude\":{},\"emdCardNumber\":\"{}\",\"time_utc\":\"{}\",\"time_local\":\"{}\",\"response_time_sec\":{}, \"day_of_week\":{},\"weekend_or_not\":{},\"geometry\":\"{}\",\"Incident_ID\":\"{}\",\"Dist_to_Seg\":{},\"XDSegID\":{}" \
              .format(json_data.iloc[i].latitude, json_data.iloc[i].longitude, json_data.iloc[i].emdCardNumber, json_data.iloc[i].time_utc, json_data.iloc[i].time_local \
                      , json_data.iloc[i].response_time_sec, json_data.iloc[i].day_of_week, json_data.iloc[i].weekend_or_not, json_data.iloc[i].geometry,json_data.iloc[i].Incident_ID \
                      , json_data.iloc[i].Dist_to_Seg, json_data.iloc[i].XDSegID)+"}"

    try:
      # Send the data to Kafka Server
      producer.send(topic, send_data.encode('utf-8'))
    except:
       print("send error")
    
    count = count + 1

  print("Total Count: ", count)
  
  

In [10]:
# execute the producer for incident project
myProducer(server_ip, "incident_NFD_XDSegID_BigDataClass.pkl", "incident")

kafka_server_addr:  34.204.95.129:9092


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


Total Count:  29765
